In [1]:
import tensorflow as tf
import numpy as np
import datetime
import random
from sklearn.model_selection import train_test_split

from data.generator import generateDataSet, saveDataSet, timedDataSet

2022-11-23 09:17:55.445156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
input1, output1, mean1, std1 = generateDataSet("./datasets/raw/logs_WT.csv", 
                                "./datasets/raw/Resultaten_WT.csv", 
                                datetime.datetime(2020, 9, 15, 0, 0).timestamp(), 
                                datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
                                50)
input2, output2, mean2, std2 = generateDataSet("./datasets/raw/logs_DN.csv", 
                                "./datasets/raw/Resultaten_DN.csv", 
                                datetime.datetime(2020, 9, 15, 0, 0).timestamp(), 
                                datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
                                50)

input3, output3, mean3, std3 = generateDataSet("./datasets/raw/logs_RS.csv", 
                                "./datasets/raw/Resultaten_DN.csv", 
                                datetime.datetime(2021, 2, 1, 0, 0).timestamp(), 
                                datetime.datetime(2021, 6, 30, 0, 0).timestamp(),
                                50)
seed1 = random.randrange(0,100)
seed2 = random.randrange(0,100)
seed3 = random.randrange(0,100)

random.Random(seed1).shuffle(input1)
random.Random(seed1).shuffle(output1)

random.Random(seed2).shuffle(input2)
random.Random(seed2).shuffle(output2)

random.Random(seed3).shuffle(input3)
random.Random(seed3).shuffle(output3)

input = []
output = []

input.extend(input1)
output.extend(output1)

input.extend(input2)
output.extend(output2)

input.extend(input3)
output.extend(output3)

x = np.array(input)
y = np.array(output)

In [13]:
print(mean1)
print(std1)

2.5502958579881656
0.6048251994878409


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

print(len(x_train))
print(len(x_test))

32212
10738


In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(np.array(input1).shape[1], 1)),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512,  activation='relu'),
    tf.keras.layers.Dense(64,   activation='relu'),
    tf.keras.layers.Dense(32,   activation='relu'),
    tf.keras.layers.Dense(16,   activation='relu'),
    tf.keras.layers.Dense(1),
])

model.summary()

model.compile(optimizer=tf.optimizers.Adam(), loss='huber', metrics=['mse', 'mae', 'mape'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 17)                0         
                                                                 
 dense_70 (Dense)            (None, 2048)              36864     
                                                                 
 dense_71 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_72 (Dense)            (None, 512)               524800    
                                                                 
 dense_73 (Dense)            (None, 64)                32832     
                                                                 
 dense_74 (Dense)            (None, 32)                2080      
                                                                 
 dense_75 (Dense)            (None, 16)              

In [21]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1007/1007 [==============================] - 4s 3ms/step - loss: 0.4457 - mse: 0.9833 - mae: 0.8233 - mape: 118.9235
Epoch 2/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.4268 - mse: 0.9244 - mae: 0.8015 - mape: 119.0252
Epoch 3/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.4156 - mse: 0.9009 - mae: 0.7875 - mape: 122.2913
Epoch 4/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.4039 - mse: 0.8755 - mae: 0.7684 - mape: 120.3784
Epoch 5/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.3946 - mse: 0.8541 - mae: 0.7507 - mape: 116.9555
Epoch 6/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.3810 - mse: 0.8231 - mae: 0.7259 - mape: 113.2751
Epoch 7/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.3708 - mse: 0.7992 - mae: 0.7051 - mape: 110.1477
Epoch 8/10
1007/1007 [==============================] - 3s 3ms/step - loss: 0.3597 - mse: 0.7748 - mae: 

In [29]:
print(np.mean(np.round(model.predict(input1) * std1 + mean1) == np.round(np.array(output1) * std1 + mean1)))
print(np.mean(np.round(model.predict(input2) * std2 + mean2) == np.round(np.array(output2) * std2 + mean2)))
print(np.mean(np.round(model.predict(input3) * std3 + mean3) == np.round(np.array(output3) * std3 + mean3)))

ValueError: in user code:

    File "/usr/lib/python3.10/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/usr/lib/python3.10/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/lib/python3.10/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/lib/python3.10/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/usr/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/lib/python3.10/site-packages/keras/engine/input_spec.py", line 216, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "sequential_10" expects 1 input(s), but it received